In [ ]:
import os
import shutil
import sys

from utils.tleap import Tleap_Preparation
from utils.molecular_dynamics import Molecular_Dynamics

In [ ]:
tleap = True

In [ ]:
if tleap == True:
    tleap_prep = Tleap_Preparation("./example_run/input/8gcy.pdb", "./example_run/input/crystal.mol2")
    tleap_prep.work_dir("./example_run/")
    tleap_prep.pdb2pqr()
    tleap_prep.obabel()
    tleap_prep.rdkit_sanitize()
    tleap_prep.create_complex()
else:
    pass

In [4]:
if tleap == True:
    tleap_prep.antechamber_ligand()
    tleap_prep.run_tleap_box(10)
    tleap_prep.run_tleap_ions(0.15)
else:
    pass

In [ ]:
if tleap == True:
    os.makedirs("./RUN", exist_ok=True)
    shutil.copy("./system.prmtop", "./RUN/system.prmtop")
    shutil.copy("./system.inpcrd", "./RUN/system.inpcrd")
else:
    pass

In [ ]:
try:
    os.chdir("./RUN")
    # Verify that the current working directory is as expected
    assert os.getcwd().endswith("RUN"), "Directory change to RUN failed"
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Run settings
delta_pico = 0.002

nvt_settings = {
    "steps": int(300 // delta_pico),      
    "dcd_save":int(50 // delta_pico),
    "log_save":int(1 // delta_pico),
    "temps_list_simulating":[50, 100, 150, 200, 250, 300, 301]
}

npt_settings = {
    "steps": int(300 // delta_pico),      
    "dcd_save": int(50 // delta_pico),
    "log_save": int(1 // delta_pico),
    "rests_list_decreasing":[1000, 100, 10, 1, 0],
    "atoms_to_restraints":{"CA"}
}

md_settings = {
    "steps": int(1000 // delta_pico),     
    "dcd_save":int(150 // delta_pico),
    "log_save":int(20 // delta_pico)
}

In [ ]:
# run_type = args.run_type
# gpu_id = str(args.gpu_id)
# rerun = args.rerun

run_type = "eq"
gpu_id = "0"
rerun = False

if run_type in ["prod", "eq"]:
    pass
else:
    print("Something Wrong")
    sys.exit()

In [ ]:
### Equilibration 
if run_type == "eq":

    md = Molecular_Dynamics("./system.prmtop", "./system.inpcrd", "0")

    # Create the System
    md.create_system()
    md.choose_integrator_params(nvt_settings["temps_list_simulating"][0], delta_pico)
    md.setup_simulation()

    # Minimize the System
    md.Minimization.minimize(md)

    ## NVT
    # Restraints the molecules of Water
    md.Nvt.restraints_water(md)

    # Choose temperature gradient
    temps = nvt_settings["temps_list_simulating"]
    partial_steps = nvt_settings["steps"] // len(temps)

    # Setup Reporters
    md.simulation.reporters.clear()
    md.Nvt.setup_reporter(md, "Step1_Nvt", nvt_settings["steps"], nvt_settings["dcd_save"], nvt_settings["log_save"], False)

    # Run NVT
    for t in temps:
        print(f"Temp = {t}")
        md.Nvt.run(md, partial_steps, t)

    ## NPT
    # Remove all previus restraints
    md.remove_all_restraints()

    # Add barostat
    md.Npt.add_barostat(md)

    # Choose restraints gradient
    restr_list = npt_settings["rests_list_decreasing"]
    partial_steps = npt_settings["steps"] // len(restr_list)

    # Setup reporters
    md.simulation.reporters.clear()
    md.Npt.setup_reporter(md, "Step2_Npt", npt_settings["steps"], npt_settings["dcd_save"], npt_settings["log_save"], False)

    # Run NPT
    for r in restr_list:
        md.Npt.restraint_backbone(md, r, npt_settings["atoms_to_restraints"])
        print(f"Restr = {r}")
        md.Npt.run(md, partial_steps)
        md.remove_all_restraints()
    
    ## Remove all restraints and save the last state
    md.remove_all_restraints()
    md.simulation.reporters.clear()
    final_npt_checkpoint = "step2_last_NVT.chk"
    md.simulation.saveCheckpoint(final_npt_checkpoint)

In [ ]:
run_type = "prod"

In [ ]:
if run_type == "prod":

    md = Molecular_Dynamics("./system.prmtop", "./system.inpcrd", gpu_id)

    # Create the System
    md.create_system()
    md.choose_integrator_params(300, 0.002)
    md.setup_simulation()

    if rerun == False:
        
        # Load the NVT checkpoint
        final_npt_checkpoint = "step2_last_NVT.chk"
        with open(final_npt_checkpoint, 'rb') as f:
            md.simulation.context.loadCheckpoint(f.read())
        
        # Setup reporters
        md.simulation.reporters.clear()
        md.Plain_Md.setup_reporter(md, f"Step3_Md_Rep{md.n_gpu}", md_settings["steps"], md_settings["dcd_save"], md_settings["log_save"], False)

        # Run MD
        md.Plain_Md.run(md, f"Step3_Md_Rep{md.n_gpu}", md_settings["steps"])
        
    elif rerun == True:
        
        final_md_checkpoint = f"Step3_Md_Rep{md.n_gpu}.chk"
        
        # Retrieve the last checkpoint
        with open(final_md_checkpoint, 'rb') as f:
                md.simulation.context.loadCheckpoint(f.read())
        
        # Setup reporters
        md.Plain_Md.setup_reporter(md, f"Step3_Md_Rep{md.n_gpu}", md_settings["steps"], md_settings["dcd_save"], md_settings["log_save"], True)
        md.Plain_Md.run(md, f"Step3_Md_Rep{md.n_gpu}", md_settings["steps"])
        
else:
    pass